In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import math

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
# initialize data

images = mnist.train.images
labels = mnist.train.labels

images = images.reshape(images.shape[0], 28,28, 1)

num_datapoints = images.shape[0]

validation_images = mnist.validation.images
validation_images = validation_images.reshape(
    validation_images.shape[0], 28, 28, 1
)
validation_labels = mnist.validation.labels


In [3]:
# define model

def run_net(images):
    net = slim.layers.conv2d(images, 32, [5,5], scope='conv1')
    net = slim.layers.max_pool2d(net, [2,2], scope='pool1')
    net = slim.layers.conv2d(net, 64, [5,5], scope='conv2')
    net = slim.layers.max_pool2d(net, [2,2], scope='pool2')
    net = slim.layers.flatten(net, scope='flatten3')
    net = slim.layers.fully_connected(net, 1024, scope='fully_connected4')
    net = slim.dropout(net, 0.5, scope='dropout5')
    net = slim.layers.fully_connected(net, 10, activation_fn=None, scope='fully_connected6')
    return net

x = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
y = tf.placeholder(tf.float32, shape=[None, 10])
predictions = run_net(x)
total_loss = tf.nn.softmax_cross_entropy_with_logits(
    logits=predictions,labels=y
)

correct = tf.equal(tf.argmax(predictions,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

learn_rate = 0.001
train_step = tf.train.AdamOptimizer(learn_rate).minimize(total_loss)

In [4]:
# train model

batch_size = 50

epochs = 1
iterations = math.ceil(epochs * num_datapoints / batch_size)
print ("datapoints: {}".format(num_datapoints))
print ("epochs: {}".format(epochs))
print ("iterations: {}".format(iterations))

sess = tf.Session()
sess.run(tf.global_variables_initializer())
for i in range(iterations):
    batch = mnist.train.next_batch(batch_size)
    batch_images = batch[0].reshape(batch_size, 28, 28, 1).tolist()
    batch_labels = batch[1]

    sess.run(train_step, feed_dict={x: batch_images, y:batch_labels})

    if i%100 == 0:
        train_accuracy = sess.run(
            accuracy,
            feed_dict={x: validation_images, y:validation_labels}
        )
        print("iteration: {}, training accuracy {}"
              .format(i, train_accuracy))


datapoints: 55000
epochs: 1
iterations: 1100
iteration: 0, training accuracy 0.17080000042915344
iteration: 100, training accuracy 0.9277999997138977
iteration: 200, training accuracy 0.9613999724388123
iteration: 300, training accuracy 0.973800003528595
iteration: 400, training accuracy 0.972599983215332
iteration: 500, training accuracy 0.9757999777793884
iteration: 600, training accuracy 0.9765999913215637
iteration: 700, training accuracy 0.9801999926567078
iteration: 800, training accuracy 0.984000027179718
iteration: 900, training accuracy 0.9801999926567078
iteration: 1000, training accuracy 0.9819999933242798


In [5]:
# evalutate test data

test_images = mnist.test.images
test_images = test_images.reshape(test_images.shape[0], 28,28,1)
test_labels = mnist.test.labels

test_accuracy = sess.run(
    accuracy,
    feed_dict={x: test_images, y:test_labels}
)

print("test accuracy: {}".format(test_accuracy))

test accuracy: 0.9840999841690063
